In [1]:
from influxdb_client import InfluxDBClient

# Define your InfluxDB connection parameters
INFLUXDB_TOKEN = "pZbpSxnkaBCa-tCQSo1p3rLv3gw6MpS1KglhS4Sv4UOANEsl1A8ngHrb9TXBg2exWgHlbczGXnvBx63sG5V2oQ=="
url = "http://206.189.139.164:8086/"
org = "test"
bucket = "wallet"

# Function to fetch trade type, trade ID, and amount
def fetch_trade_data(trade_id):
    try:
        # Initialize InfluxDB client
        client = InfluxDBClient(url=url, token=INFLUXDB_TOKEN, org=org)

        # Query API
        query_api = client.query_api()

        # Define the Flux query to fetch trade type, trade ID, and amount from the InfluxDB bucket
        flux_query = f'''
        from(bucket: "{bucket}")
          |> range(start: -30d) // Adjust the time range if necessary
          |> filter(fn: (r) => r._measurement == "wallet")
          |> filter(fn: (r) => r.tradeId == "{trade_id}") // Filter by trade ID
          |> filter(fn: (r) => r._field == "amount" or r._field == "direction") // Fetch amount and trade type
          |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
          |> keep(columns: ["_time", "tradeId", "direction", "amount"]) // Keep only necessary columns
        '''

        # Execute the query and get the results
        result = query_api.query(org=org, query=flux_query)

        # Process the results
        trades = []
        for table in result:
            for record in table.records:
                trade_data = {
                    "time": record.get_time(),
                    "tradeId": record.values.get("tradeId"),
                    "direction": record.values.get("direction"),  # Trade type: buy/sell
                    "amount": record.values.get("amount"),  # Amount involved in the trade
                }
                trades.append(trade_data)

        # Return fetched trades
        return trades

    except Exception as e:
        print(f"Error fetching trade data: {e}")

# Example usage:
trade_id = "example_trade_id"  # Replace with your trade ID
trades = fetch_trade_data(trade_id)

# Display the fetched trade data
if trades:
    for trade in trades:
        print(f"Trade ID: {trade['tradeId']}, Type: {trade['direction']}, Amount: {trade['amount']}, Time: {trade['time']}")
else:
    print("No trade data found.")


No trade data found.
